# Regex Labeling

## Data Loading

In [0]:
import pandas as pd
import numpy as np
import re

In [0]:
KK_PATH = '/content/'
DATASET_PATH = 'code_blocks_final_clean.csv' ## CODE_COLUMN = 'code_block'
# DATASET_PATH = 'chunks_30_final.csv' ## CODE_COLUMN = 'code'
# DATASET_PATH = 'nl2ml preprocessing (united).csv' ## CODE_COLUMN = 'Code'
NEW_DATASET_PATH = DATASET_PATH[:-4] + '_regex.csv'
CODE_COLUMN = 'code_block'

In [0]:
# df = pd.read_csv(KK_PATH + 'code_blocks.csv', sep='\t')
# df = df.dropna()
# df = df.drop_duplicates()
# df = df.reset_index(drop=True)
# df['code_block_length'] = np.zeros(len(df))
# for i in range(len(df)):
#     df['code_block_length'][i] = len(df['code_block'][i])
#     print(str(i)+'\r', end='')
# df = df.drop(df[df['code_block_length'] > 512].index)
# df.shape
# df.to_csv(KK_PATH + 'code_blocks_prepared.csv', sep='\t', index=False)

In [0]:
# df = pd.read_csv(KK_PATH + 'code_blocks_prepared.csv', sep='\t')
# df.shape

In [0]:
df = pd.read_csv(KK_PATH + DATASET_PATH, sep=',')

In [6]:
df.shape

(221803, 3)

In [0]:
df = df.dropna(subset=[CODE_COLUMN]).reset_index()

In [0]:
def wordListToFreqDict(wordlist):
    def sortFreqDict(freqdict):
        aux = [(freqdict[key], key) for key in freqdict]
        aux.sort()
        aux.reverse()
        return aux
    wordfreq = [wordlist.count(p) for p in wordlist]
    return sortFreqDict(dict(list(zip(wordlist,wordfreq))))

In [0]:
# %%time
# wordListToFreqDict(df.code.to_string())

In [0]:
# %%time
# nl2ml = pd.read_csv(KK_PATH + 'nl2ml.csv')
# nl2ml = nl2ml.rename({'':'code_block', '':'method_tag'})
# nl2ml_vis = nl2ml[nl2ml['method_tag'] == 'Visualization']
# tokens_visualization = wordListToFreqDict(nl2ml_vis.code_block.to_string())

In [11]:
df[CODE_COLUMN][0]

'from mpl_toolkits.mplot3d import Axes3D\nfrom sklearn.preprocessing import StandardScaler\nimport matplotlib.pyplot as plt \nimport numpy as np \nimport os \nimport pandas as pd \n,metadata:{collapsed:false,_kg_hide-input:false},'

In [0]:
def tokens_search(df, tokens, new_column_name):
    df[new_column_name] = 0
    for i in range(len(df)):
        percents = str(round(100*i/len(df),1))
        print(percents + '%\r', end='')
        row = df[CODE_COLUMN][i]
        for token in tokens:
            result = re.search(token, row)
            if result!=None:
                df[new_column_name][i] = 1
                break
    return df

## Regex Labels

In [0]:
graph = {'imports':['import', 'from']#, 'as']
          ,'load_data':['load', 'csv', 'glob', 'pickle', 'read', 'download', 'txt']
          ,'preprocessing':['preprocess', 'plot', 'heatmap', 'figure', 'encode', 'fill', 'merge', 'vectorize', 'generation', 'augmentation', 'stemming', 'lemmatization', 'scale', 'normalize', 'crop', 'convert', 'resize']
          ,'model':['LinearRegression', 'RandomForest', 'Ridge', 'Lasso', 'SGD', 'LogisticRegression', 'SVM', 'SVC', 'Layer', 'xgboost', 'LGBM', 'CatBoost']
          ,'train': ['train', 'fit', 'epoch', 'loss', 'learn', 'optimizer']
          ,'predict':['predict', 'error', 'evaluation', 'MSE', 'Test']
}

In [0]:
%%time
for i in range(0, len(graph)):
    print(i)
    vertice = list(graph.keys())[i]
    tokens = graph[vertice]
    df = tokens_search(df, tokens, vertice)

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


1
2
3
4


In [0]:
df.to_csv(NEW_DATASET_PATH, index=False)

In [0]:
# for col in list(graph.keys()):
#     display(df[df['Data format'] == 'Table'][col].value_counts())

#### Tag: Import-Output

In [0]:
# # TAG: input_output
# tokens_io = ['read', 'csv' , 'sql' , 'json' , 'png' , 'jpg' ,
#           'tsv' , 'write' , 'open' , 'print' , 'output' ,
#           'stdin' , 'stdout' , 'path' , 'dir'
# #            , 'import'
#           ]

In [0]:
# %%time
# df = tokens_search(df, tokens_io, 'tag_import_output')

In [0]:
# df.to_csv(KK_PATH + 'chunks_30_tag_io.csv', index=False)

#### Tag: Visualization

In [0]:
# tokens_visualization = ['plot', 'hist', 'sns.', 'plt.']

In [0]:
# %%time
# df = tokens_search(df, tokens_visualization, 'tag_visualization')

In [0]:
# df.to_csv(KK_PATH + 'chunks_30_tags.csv', index=False)

### TF-IDF + Log Reg

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
df = pd.read_csv(NEW_DATASET_PATH)

In [0]:
df.shape

In [0]:
corpus = df[CODE_COLUMN]
test_size = 0.1
test_rows = round(df.shape[0]*test_size)
train_rows = df.shape[0] - test_rows
train_corpus = df[CODE_COLUMN][0:test_rows]
test_corpus = df[CODE_COLUMN][train_rows:]
tag_to_predict = 'preprocessing'

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=5
                        , max_df = 0.3
                        , ngram_range = (1,2)
                        , smooth_idf = True
                       )
features = tfidf.fit_transform(corpus)
# for_pred = tfidf.transform(test_corpus)

In [0]:
%%time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(features[0:len(df[tag_to_predict])], df[tag_to_predict], test_size=0.25)
clf = LogisticRegression(random_state=4321).fit(X_train, y_train)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = clf.score(X_test, y_test)
print(f'Mean Accuracy {round(accuracy*100, 2)}%')
print(f'F1-score {round(metrics.f1_score(y_pred, y_test)*100, 2)}%')

In [0]:
metrics.plot_precision_recall_curve(clf, X_test, y_test)

In [0]:
metrics.plot_confusion_matrix(clf, X_test, y_test, values_format='d')